In [35]:
%autosave 0
import numpy 
import matplotlib
import pickle
import pandas as pd
import numpy as np 
import tensorflow as tf
from pandas import read_csv
from matplotlib import pyplot
from pandas.plotting import scatter_matrix 
from numpy import set_printoptions 
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.compose import ColumnTransformer

set_printoptions(precision=0)
pd.set_option("display.precision", 2)
pd.option_context('display.float_format', '{:0f}'.format);

seed = 15
tf.random.set_seed(seed)
numpy.random.seed(seed)

Autosave disabled


In [4]:
# Importation du fichier csv dans un dataframe
df_train_full = read_csv( './dataset-template.csv', index_col=0)
#df_train_full = read_csv( './csv/realstate-data.csv' )

In [5]:
print(df_train_full.shape)
df_train_full.head(5)

(8, 11)


,city,county,district,area_m2,ground_m2,nb_room,nb_bedroom,pool,cellar,garage,output
ref,,,,,,,,,,,
654a-123,Bordeaux,Gironde,Nouvelle Aquitaine,47,47,1,0,0,0,0,120000
654a-124,Bègles,Gironde,Nouvelle Aquitaine,54,54,2,1,0,1,0,130000
654a-125,Bordeaux,Gironde,Nouvelle Aquitaine,38,38,1,0,0,1,1,75000
654a-126,Bordeaux,Gironde,Nouvelle Aquitaine,42,42,1,0,0,0,0,89000
654a-127,Mérignac,Gironde,Nouvelle Aquitaine,71,71,3,2,0,0,0,220000


In [6]:
df_train_full.describe()

,area_m2,ground_m2,nb_room,nb_bedroom,pool,cellar,garage,output
count,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00
mean,51.62,51.62,1.88,0.75,0.12,0.50,0.25,126875.00
std,14.03,14.03,1.13,0.89,0.35,0.53,0.46,49501.62
min,38.00,38.00,1.00,0.00,0.00,0.00,0.00,75000.00
25%,42.00,42.00,1.00,0.00,0.00,0.00,0.00,92750.00
50%,45.50,45.50,1.50,0.50,0.00,0.50,0.00,113500.00
75%,58.25,58.25,2.25,1.25,0.00,1.00,0.25,142500.00
max,75.00,75.00,4.00,2.00,1.00,1.00,1.00,220000.00


In [7]:
Y_trainfull = df_train_full['output']
X_trainfull = df_train_full.drop('output', axis=1)

In [8]:
print(Y_trainfull.shape)
print(X_trainfull.shape)

(8,)
(8, 10)


# Traitement des variables catégorielles

In [13]:
cat_columns=['city', 'county', 'district']

In [16]:
categorical_pipe = Pipeline([
('onehot', OneHotEncoder())
])

In [19]:
preprocess_pipe = ColumnTransformer([
('cat', categorical_pipe, cat_columns)
])

# Choix du modèle et intégration dans la pipeline

In [22]:
model = LinearRegression()

In [23]:
full_pipe = Pipeline(
[('pp', preprocess_pipe),
('reg', model)]
)

# Séparations des données en ensemble d'entraînement et de validation

In [24]:
X_train, X_val, Y_train, Y_val = train_test_split(X_trainfull, Y_trainfull, random_state = seed)

In [25]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(6, 10)
(6,)
(2, 10)
(2,)


# Création modèle

In [28]:
 full_pipe.fit(X_train, Y_train);

In [29]:
y_train_pred = full_pipe.predict(X_train)
y_val_pred = full_pipe.predict(X_val)

In [34]:
mae_train = round(metrics.mean_absolute_error(Y_train, y_train_pred), 0)
mae_val = round(metrics.mean_absolute_error(Y_val, y_val_pred), 0)
print('MAE sur train : ', mae_train, '€')
print('MAE sur val : ', mae_val, '€')

MAE sur train :  24833.0 €
MAE sur val :  38000.0 €


In [37]:
pickle.dump(model, open( "model.file", "wb" ))